# Correlation Analysis of Census Data and Crime

In [2]:
import pandas as pd

# Load the CSV file
file_path = 'data/combined_crime_data.csv'
crime_data = pd.read_csv(file_path)

# Define the COVID-19 period as February 2020 to March 2021
covid_columns = [f'{year}{month:02d}' for year in range(2020, 2022) 
                 for month in range(1, 13) if (year == 2020 and month >= 2) or (year == 2021 and month <= 3)]

# Remove the COVID-19 columns from the dataset
crime_data_filtered = crime_data.drop(columns=covid_columns, errors='ignore')

crime_data_filtered.head()


,WardCode,WardName,LookUp_BoroughName,201401,201402,201403,201404,201405,201406,201407,...,202310,202311,202312,202401,202402,202403,202404,202405,202406,202407
0,E05009317,Bethnal Green,Tower Hamlets,151,142,169,175,168,146,165,...,213,217,228,175,240,204,197,201,188,175
1,E05009318,Blackwall & Cubitt Town,Tower Hamlets,58,55,56,67,63,61,72,...,144,103,137,162,140,112,121,128,131,100
2,E05009319,Bow East,Tower Hamlets,102,119,132,122,149,158,131,...,177,159,208,135,148,121,169,181,148,138
3,E05009320,Bow West,Tower Hamlets,68,74,89,84,101,258,188,...,128,119,143,97,105,118,120,150,87,96
4,E05009321,Bromley North,Tower Hamlets,65,70,88,56,68,79,62,...,112,134,134,115,116,113,97,116,96,123


In [7]:
# Aggregate the crime data by ward
crime_aggregated = crime_data_filtered.groupby('WardCode').sum().reset_index()
crime_aggregated['TotalCrime'] = crime_aggregated.iloc[:, 3:].sum(axis=1)

# Keep only WardCode and TotalCrime for simplicity
crime_aggregated = crime_aggregated[['WardCode', 'TotalCrime']]

print(crime_aggregated.head())


    WardCode  TotalCrime
0  E05009317       22649
1  E05009318       11583
2  E05009319       16965
3  E05009320       13643
4  E05009321       10258


### Accommodation Data 

In [3]:
import pandas as pd

accommodation_data = pd.read_csv('data/Census/Accomodation Type.csv')

# Step 2: Pivot the data so that accommodation types become columns and their counts become values
accommodation_pivot = accommodation_data.pivot_table(
    index=['Electoral wards and divisions Code', 'Electoral wards and divisions'],  
    columns='Accommodation type (8 categories)', 
    values='Observation',  
    aggfunc='sum'  
).reset_index()

# Rename columns for clarity
accommodation_pivot.columns.name = None
accommodation_pivot.columns = ['WardCode', 'WardName'] + [f'AccomType_{col}' for col in accommodation_pivot.columns[2:]]

print(accommodation_pivot.head())

    WardCode                 WardName  \
0  E05009317            Bethnal Green   
1  E05009318  Blackwall & Cubitt Town   
2  E05009319                 Bow East   
3  E05009320                 Bow West   
4  E05009321            Bromley North   

   AccomType_A caravan or other mobile or temporary structure  \
0                                                  5            
1                                                 64            
2                                                 10            
3                                                  9            
4                                                 14            

   AccomType_Detached  \
0                  58   
1                  36   
2                 124   
3                  60   
4                  49   

   AccomType_In a commercial building, for example, in an office building, hotel or over a shop  \
0                                                 74                                              
1            

In [11]:
import pandas as pd

#Merge the crime and accommodation data by WardCode
merged_data = pd.merge(crime_aggregated, accommodation_pivot, on='WardCode', how='inner')

# Select columns for correlation analysis (excluding WardName)
numeric_columns = merged_data.select_dtypes(include=['float64', 'int64'])

# Calculate Pearson correlation (linear relationships)
pearson_corr = numeric_columns.corr(method='pearson')
pearson_crime_accommodation_correlation = pearson_corr['TotalCrime'].drop('TotalCrime')

# Calculate Spearman correlation (non-linear relationships)
spearman_corr = numeric_columns.corr(method='spearman')
spearman_crime_accommodation_correlation = spearman_corr['TotalCrime'].drop('TotalCrime')

# Display both Pearson and Spearman correlations
correlation_results = pd.DataFrame({
    'Pearson (Linear)': pearson_crime_accommodation_correlation,
    'Spearman (Non-linear)': spearman_crime_accommodation_correlation
})

print(correlation_results)


                                                    Pearson (Linear)  \
AccomType_A caravan or other mobile or temporar...          0.045960   
AccomType_Detached                                         -0.157087   
AccomType_In a commercial building, for example...          0.556050   
AccomType_In a purpose-built block of flats or ...          0.369290   
AccomType_Part of a converted or shared house, ...          0.079019   
AccomType_Part of another converted building, f...          0.285672   
AccomType_Semi-detached                                    -0.189998   
AccomType_Terraced                                         -0.079149   

                                                    Spearman (Non-linear)  
AccomType_A caravan or other mobile or temporar...               0.382661  
AccomType_Detached                                              -0.267542  
AccomType_In a commercial building, for example...               0.507891  
AccomType_In a purpose-built block of flats or 

### Economic Activity

In [15]:
import pandas as pd

economic_activity_data = pd.read_csv('data/Census/Economic Activity.csv')

# Pivot the economic activity data to reshape it
economic_activity_pivot = economic_activity_data.pivot_table(
    index=['Electoral wards and divisions Code', 'Electoral wards and divisions'], 
    columns='Economic activity status (20 categories)',  
    values='Observation',  
    aggfunc='sum' 
).reset_index()

# Rename the columns for clarity
economic_activity_pivot.columns.name = None
economic_activity_pivot.columns = ['WardCode', 'WardName'] + [f'EconActivity_{col}' for col in economic_activity_pivot.columns[2:]]

print(economic_activity_pivot.head())


    WardCode                 WardName  EconActivity_Does not apply  \
0  E05009317            Bethnal Green                         4006   
1  E05009318  Blackwall & Cubitt Town                         3249   
2  E05009319                 Bow East                         3224   
3  E05009320                 Bow West                         2396   
4  E05009321            Bromley North                         2479   

   EconActivity_Economically active (excluding full-time students): In employment: Employee: Full-time  \
0                                               5619                                                     
1                                               9159                                                     
2                                               7413                                                     
3                                               4599                                                     
4                                               3

In [17]:
# Merge the crime and economic activity data by WardCode
merged_economic_data = pd.merge(crime_aggregated, economic_activity_pivot, on='WardCode', how='inner')

# Select columns for correlation analysis 
numeric_columns_economic = merged_economic_data.select_dtypes(include=['float64', 'int64'])

# Calculate Pearson correlation (linear relationships)
pearson_corr_economic = numeric_columns_economic.corr(method='pearson')
pearson_crime_economic_correlation = pearson_corr_economic['TotalCrime'].drop('TotalCrime')

# Calculate Spearman correlation (non-linear relationships)
spearman_corr_economic = numeric_columns_economic.corr(method='spearman')
spearman_crime_economic_correlation = spearman_corr_economic['TotalCrime'].drop('TotalCrime')

# Display both Pearson and Spearman correlations
correlation_economic_results = pd.DataFrame({
    'Pearson (Linear)': pearson_crime_economic_correlation,
    'Spearman (Non-linear)': spearman_crime_economic_correlation
})

print(correlation_economic_results)


                                                    Pearson (Linear)  \
EconActivity_Does not apply                                -0.011254   
EconActivity_Economically active (excluding ful...          0.166356   
EconActivity_Economically active (excluding ful...          0.059138   
EconActivity_Economically active (excluding ful...          0.089987   
EconActivity_Economically active (excluding ful...          0.104877   
EconActivity_Economically active (excluding ful...          0.065483   
EconActivity_Economically active (excluding ful...          0.083611   
EconActivity_Economically active (excluding ful...          0.221127   
EconActivity_Economically active and a full-tim...          0.239681   
EconActivity_Economically active and a full-tim...          0.224025   
EconActivity_Economically active and a full-tim...          0.064279   
EconActivity_Economically active and a full-tim...          0.153034   
EconActivity_Economically active and a full-tim...          0.07

### Employment History

In [21]:
import pandas as pd
employment_history_data = pd.read_csv('data/Census/Employment History.csv')
# Pivot the employment history data to reshape it
employment_history_pivot = employment_history_data.pivot_table(
    index=['Electoral wards and divisions Code', 'Electoral wards and divisions'], 
    columns='Employment history (4 categories)', 
    values='Observation',  
    aggfunc='sum' 
).reset_index()

# Rename the columns for clarity
employment_history_pivot.columns.name = None
employment_history_pivot.columns = ['WardCode', 'WardName'] + [f'EmpHistory_{col}' for col in employment_history_pivot.columns[2:]]

# Merge the crime data with the employment history data by WardCode
merged_employment_data = pd.merge(crime_aggregated, employment_history_pivot, on='WardCode', how='inner')

# Select columns for correlation analysis 
numeric_columns_employment = merged_employment_data.select_dtypes(include=['float64', 'int64'])

# Calculate Pearson correlation (linear relationships)
pearson_corr_employment = numeric_columns_employment.corr(method='pearson')
pearson_crime_employment_correlation = pearson_corr_employment['TotalCrime'].drop('TotalCrime')

# Calculate Spearman correlation (non-linear relationships)
spearman_corr_employment = numeric_columns_employment.corr(method='spearman')

# Extracting Spearman correlations with TotalCrime
spearman_crime_employment_correlation = spearman_corr_employment['TotalCrime'].drop('TotalCrime')

# Combine both Pearson and Spearman correlations
correlation_employment_results = pd.DataFrame({
    'Pearson (Linear)': pearson_crime_employment_correlation,
    'Spearman (Non-linear)': spearman_crime_employment_correlation
})

print(correlation_employment_results)


                                                    Pearson (Linear)  \
EmpHistory_Does not apply                                   0.113471   
EmpHistory_Not in employment: Never worked                  0.244306   
EmpHistory_Not in employment: Not worked in the...          0.016690   
EmpHistory_Not in employment: Worked in the las...          0.326984   

                                                    Spearman (Non-linear)  
EmpHistory_Does not apply                                        0.462677  
EmpHistory_Not in employment: Never worked                       0.604999  
EmpHistory_Not in employment: Not worked in the...               0.151352  
EmpHistory_Not in employment: Worked in the las...               0.709305  


### General Health

In [25]:
import pandas as pd
general_health_data= pd.read_csv('data/Census/General Health.csv')

# Pivot the general health data to reshape it
general_health_pivot = general_health_data.pivot_table(
    index=['Electoral wards and divisions Code', 'Electoral wards and divisions'],
    columns='General health (6 categories)',  
    values='Observation',  
    aggfunc='sum'  
).reset_index()

# Rename the columns for clarity
general_health_pivot.columns.name = None
general_health_pivot.columns = ['WardCode', 'WardName'] + [f'Health_{col}' for col in general_health_pivot.columns[2:]]

# Aggregate 'Good health' and 'Very good health', and 'Bad health' and 'Very bad health'
general_health_pivot['Health_Good_VeryGood'] = general_health_pivot['Health_Good health'] + general_health_pivot['Health_Very good health']
general_health_pivot['Health_Bad_VeryBad'] = general_health_pivot['Health_Bad health'] + general_health_pivot['Health_Very bad health']

general_health_pivot = general_health_pivot.drop(columns=['Health_Good health', 'Health_Very good health', 'Health_Bad health', 'Health_Very bad health'])

# Merge the crime data with the aggregated general health data 
merged_health_data = pd.merge(crime_aggregated, general_health_pivot, on='WardCode', how='inner')

# Select columns for correlation analysis 
numeric_columns_health = merged_health_data.select_dtypes(include=['float64', 'int64'])

# Calculate Pearson correlation (linear relationships)
pearson_corr_health = numeric_columns_health.corr(method='pearson')
pearson_crime_health_correlation = pearson_corr_health['TotalCrime'].drop('TotalCrime')

# Calculate Spearman correlation (non-linear relationships)
spearman_corr_health = numeric_columns_health.corr(method='spearman')
spearman_crime_health_correlation = spearman_corr_health['TotalCrime'].drop('TotalCrime')

# Combine both Pearson and Spearman correlations
correlation_health_results = pd.DataFrame({
    'Pearson (Linear)': pearson_crime_health_correlation,
    'Spearman (Non-linear)': spearman_crime_health_correlation
})


print(correlation_health_results)


                       Pearson (Linear)  Spearman (Non-linear)
Health_Does not apply               NaN                    NaN
Health_Fair health             0.110795               0.436623
Health_Good_VeryGood           0.148743               0.485393
Health_Bad_VeryBad             0.172677               0.555463


### Household Composition

In [27]:
import pandas as pd
household_composition_data=pd.read_csv('data/Census/Household Composition.csv')
# Pivot the household composition data to reshape it
household_composition_pivot = household_composition_data.pivot_table(
    index=['Electoral wards and divisions Code', 'Electoral wards and divisions'],  
    columns='Household composition (15 categories)',  
    values='Observation', 
    aggfunc='sum'  
).reset_index()

# Rename the columns for clarity
household_composition_pivot.columns.name = None
household_composition_pivot.columns = ['WardCode', 'WardName'] + [f'HouseholdComp_{col}' for col in household_composition_pivot.columns[2:]]

# Merge the crime data with the household composition data 
merged_household_data = pd.merge(crime_aggregated, household_composition_pivot, on='WardCode', how='inner')

# Select columns for correlation analysis 
numeric_columns_household = merged_household_data.select_dtypes(include=['float64', 'int64'])

# Calculate Pearson correlation (linear relationships)
pearson_corr_household = numeric_columns_household.corr(method='pearson')
pearson_crime_household_correlation = pearson_corr_household['TotalCrime'].drop('TotalCrime')

# Calculate Spearman correlation (non-linear relationships)
spearman_corr_household = numeric_columns_household.corr(method='spearman')
spearman_crime_household_correlation = spearman_corr_household['TotalCrime'].drop('TotalCrime')

# Combine both Pearson and Spearman correlations
correlation_household_results = pd.DataFrame({
    'Pearson (Linear)': pearson_crime_household_correlation,
    'Spearman (Non-linear)': spearman_crime_household_correlation
})

print(correlation_household_results)


                                                    Pearson (Linear)  \
HouseholdComp_Does not apply                                     NaN   
HouseholdComp_One-person household: Aged 66 yea...          0.008888   
HouseholdComp_One-person household: Other                   0.442459   
HouseholdComp_Other household types: Other rela...          0.421213   
HouseholdComp_Other household types: Other, inc...          0.279913   
HouseholdComp_Other household types: With depen...          0.024604   
HouseholdComp_Single family household: All aged...         -0.185024   
HouseholdComp_Single family household: Cohabiti...         -0.049522   
HouseholdComp_Single family household: Cohabiti...          0.214365   
HouseholdComp_Single family household: Cohabiti...         -0.024693   
HouseholdComp_Single family household: Lone par...          0.130976   
HouseholdComp_Single family household: Lone par...          0.137617   
HouseholdComp_Single family household: Married ...         -0.15

### Housing Density

In [29]:
import pandas as pd
housing_density_data= pd.read_csv('data/Census/Housing Density.csv')
# Merge the crime data with the housing density data by WardCode
merged_housing_density_data = pd.merge(crime_aggregated, housing_density_data, 
                                       left_on='WardCode', right_on='Electoral wards and divisions Code', how='inner')

# Select columns for correlation analysis 
numeric_columns_housing_density = merged_housing_density_data.select_dtypes(include=['float64', 'int64'])

# Calculate Pearson correlation (linear relationships)
pearson_corr_housing_density = numeric_columns_housing_density.corr(method='pearson')
pearson_crime_housing_density_correlation = pearson_corr_housing_density['TotalCrime'].drop('TotalCrime')

# Calculate Spearman correlation (non-linear relationships)
spearman_corr_housing_density = numeric_columns_housing_density.corr(method='spearman')
spearman_crime_housing_density_correlation = spearman_corr_housing_density['TotalCrime'].drop('TotalCrime')

# Combine both Pearson and Spearman correlations
correlation_housing_density_results = pd.DataFrame({
    'Pearson (Linear)': pearson_crime_housing_density_correlation,
    'Spearman (Non-linear)': spearman_crime_housing_density_correlation
})

print(correlation_housing_density_results)


             Pearson (Linear)  Spearman (Non-linear)
Observation          0.240263               0.556178


### Occupancy Rating

In [31]:
import pandas as pd
occupancy_rating_data= pd.read_csv('data/Census/Occupancy Rating.csv')
# Step 1: Pivot the occupancy rating data to reshape it
occupancy_rating_pivot = occupancy_rating_data.pivot_table(
    index=['Electoral wards and divisions Code', 'Electoral wards and divisions'], 
    columns='Occupancy rating for bedrooms (6 categories)',  
    values='Observation', 
    aggfunc='sum' 
).reset_index()

# Rename the columns for clarity
occupancy_rating_pivot.columns.name = None
occupancy_rating_pivot.columns = ['WardCode', 'WardName'] + [f'OccupancyRating_{col}' for col in occupancy_rating_pivot.columns[2:]]

# Merge the crime data with the occupancy rating data by WardCode
merged_occupancy_data = pd.merge(crime_aggregated, occupancy_rating_pivot, on='WardCode', how='inner')

# Select columns for correlation analysis 
numeric_columns_occupancy = merged_occupancy_data.select_dtypes(include=['float64', 'int64'])

# Calculate Pearson correlation (linear relationships)
pearson_corr_occupancy = numeric_columns_occupancy.corr(method='pearson')
pearson_crime_occupancy_correlation = pearson_corr_occupancy['TotalCrime'].drop('TotalCrime')

# Calculate Spearman correlation (non-linear relationships)
spearman_corr_occupancy = numeric_columns_occupancy.corr(method='spearman')
spearman_crime_occupancy_correlation = spearman_corr_occupancy['TotalCrime'].drop('TotalCrime')

# Combine both Pearson and Spearman correlations
correlation_occupancy_results = pd.DataFrame({
    'Pearson (Linear)': pearson_crime_occupancy_correlation,
    'Spearman (Non-linear)': spearman_crime_occupancy_correlation
})
print(correlation_occupancy_results)


                                                    Pearson (Linear)  \
OccupancyRating_Does not apply                                   NaN   
OccupancyRating_Occupancy rating of bedrooms: +1            0.113972   
OccupancyRating_Occupancy rating of bedrooms: +...         -0.191211   
OccupancyRating_Occupancy rating of bedrooms: -1            0.225017   
OccupancyRating_Occupancy rating of bedrooms: -...          0.150648   
OccupancyRating_Occupancy rating of bedrooms: 0             0.389986   

                                                    Spearman (Non-linear)  
OccupancyRating_Does not apply                                        NaN  
OccupancyRating_Occupancy rating of bedrooms: +1                 0.287993  
OccupancyRating_Occupancy rating of bedrooms: +...              -0.316086  
OccupancyRating_Occupancy rating of bedrooms: -1                 0.644565  
OccupancyRating_Occupancy rating of bedrooms: -...               0.540819  
OccupancyRating_Occupancy rating of bed

### Qualifications

In [41]:
import pandas as pd
qualifications_data= pd.read_csv('data/Census/Qualifications.csv')
# Pivot the qualifications data to reshape it
qualifications_pivot = qualifications_data.pivot_table(
    index=['Electoral wards and divisions Code', 'Electoral wards and divisions'],  # Keep ward code and ward name as index
    columns='Highest level of qualification (8 categories)',  # Make the qualification categories the new columns
    values='Observation',  # Use 'Observation' as the value for each category
    aggfunc='sum'  # In case there are duplicates, sum the values
).reset_index()

# Rename the columns for clarity
qualifications_pivot.columns.name = None
qualifications_pivot.columns = ['WardCode', 'WardName'] + [f'Qualifications_{col}' for col in qualifications_pivot.columns[2:]]

# Select only "No qualifications" and "Other qualifications"
qualifications_filtered = qualifications_pivot[['WardCode', 'Qualifications_No qualifications'
                                                ]]

# Merge the crime data with the filtered qualifications data by WardCode
merged_qualifications_filtered = pd.merge(crime_aggregated, qualifications_filtered, on='WardCode', how='inner')

# Select only numeric columns for correlation analysis
numeric_columns_qualifications_filtered = merged_qualifications_filtered.select_dtypes(include=['float64', 'int64'])

# Calculate Pearson correlation (linear relationships)
pearson_corr_qualifications_filtered = numeric_columns_qualifications_filtered.corr(method='pearson')
pearson_crime_qualifications_filtered_correlation = pearson_corr_qualifications_filtered['TotalCrime'].drop('TotalCrime')

# Calculate Spearman correlation (non-linear relationships)
spearman_corr_qualifications_filtered = numeric_columns_qualifications_filtered.corr(method='spearman')
spearman_crime_qualifications_filtered_correlation = spearman_corr_qualifications_filtered['TotalCrime'].drop('TotalCrime')

# Combine both Pearson and Spearman correlations
correlation_qualifications_filtered_results = pd.DataFrame({
    'Pearson (Linear)': pearson_crime_qualifications_filtered_correlation,
    'Spearman (Non-linear)': spearman_crime_qualifications_filtered_correlation
})

print(correlation_qualifications_filtered_results)


                                  Pearson (Linear)  Spearman (Non-linear)
Qualifications_No qualifications          0.093103               0.407643


### Tenure

In [43]:
import pandas as pd
tenure_data= pd.read_csv('data/Census/Tenure.csv')
# Pivot the tenure data to reshape it
tenure_pivot = tenure_data.pivot_table(
    index=['Electoral wards and divisions Code', 'Electoral wards and divisions'],  # Keep ward code and ward name as index
    columns='Tenure of household (9 categories)',  # Make the tenure categories the new columns
    values='Observation',  # Use 'Observation' as the value for each category
    aggfunc='sum'  # In case there are duplicates, sum the values
).reset_index()

# Rename the columns for clarity
tenure_pivot.columns.name = None
tenure_pivot.columns = ['WardCode', 'WardName'] + [f'Tenure_{col}' for col in tenure_pivot.columns[2:]]

# Merge the crime data with the tenure data by WardCode
merged_tenure_data = pd.merge(crime_aggregated, tenure_pivot, on='WardCode', how='inner')

# Select only numeric columns for correlation analysis (excluding WardName)
numeric_columns_tenure = merged_tenure_data.select_dtypes(include=['float64', 'int64'])

# Calculate Pearson correlation (linear relationships)
pearson_corr_tenure = numeric_columns_tenure.corr(method='pearson')
pearson_crime_tenure_correlation = pearson_corr_tenure['TotalCrime'].drop('TotalCrime')

# Calculate Spearman correlation (non-linear relationships)
spearman_corr_tenure = numeric_columns_tenure.corr(method='spearman')
spearman_crime_tenure_correlation = spearman_corr_tenure['TotalCrime'].drop('TotalCrime')

# Combine both Pearson and Spearman correlations
correlation_tenure_results = pd.DataFrame({
    'Pearson (Linear)': pearson_crime_tenure_correlation,
    'Spearman (Non-linear)': spearman_crime_tenure_correlation
})

print(correlation_tenure_results)


                                                    Pearson (Linear)  \
Tenure_Does not apply                                            NaN   
Tenure_Lives rent free                                      0.208829   
Tenure_Owned: Owns outright                                -0.150911   
Tenure_Owned: Owns with a mortgage or loan                 -0.163201   
Tenure_Private rented: Other private rented                 0.482460   
Tenure_Private rented: Private landlord or lett...          0.364467   
Tenure_Shared ownership: Shared ownership                   0.168430   
Tenure_Social rented: Other social rented                   0.284070   
Tenure_Social rented: Rents from council or Loc...          0.173268   

                                                    Spearman (Non-linear)  
Tenure_Does not apply                                                 NaN  
Tenure_Lives rent free                                           0.526293  
Tenure_Owned: Owns outright                        